In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
sys.path.append('..')

import utilities.functions as functions


from utilities.functions import (
    summary,
    matriz_migracao,
    save_parquet,
    gerar_resumo_decis,
    resumo_coorte_ativa,
    cria_base_decil_wide

)

In [ ]:
df_pub_u = pd.read_parquet("../../../dados/gold/df_pub_un.parquet")

In [ ]:
df_pub_u.head()

Sumarizacao na visao cliente - mes

In [ ]:
summary(df_pub_u)

Clientes mes x target

In [ ]:
clientes_mes = (df_pub_u.groupby(['order_created_month', 'is_target'])
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos'))
clientes_mes

 Matriz de migracao - cliente

In [ ]:
matriz_migracao(df_pub_u,mes_0=12,mes_1=1)

Dado o montante de clientes novos em janeiro a analise sera feita basead em cliente ambos os meses e cliente somente em janeiro

In [ ]:
id_both_monht=df_pub_u[df_pub_u['order_created_month']==12]['customer_id'].unique()
publico_janeiro_dezembro = df_pub_u[df_pub_u['customer_id'].isin(id_both_monht)].reset_index(drop=True)
publico_janeiro = df_pub_u[~df_pub_u['customer_id'].isin(id_both_monht)].reset_index(drop=True)

In [ ]:
publico_janeiro.head()

In [ ]:
save_parquet(publico_janeiro_dezembro, "gold", "publico_janeiro_dezembro.parquet")


Pedidos por mes - target

In [ ]:
pedidos_total = (
    publico_janeiro_dezembro.groupby(['is_target','order_created_month'])['num_pedidos_mes']
      .sum()
      .reset_index(name='numero_de_pedidos_total')
)

pedidos_total

Distruibuicao do total amount por mes

In [ ]:
df_stats_mes = publico_janeiro_dezembro.groupby(['order_created_month', 'is_target'])['total_amount_mes'].agg(
    Média=('mean'),
    Mediana=('median'),
    Mínimo=('min'),
    Máximo=('max'),
    Desvio_Padrão=('std')
).round(2)
df_stats_mes

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(
    data=publico_janeiro_dezembro,
    y='total_amount_mes',
    x='order_created_month',  
    hue='is_target',          
    
)

plt.title('Distribuição do Valor Gasto por Mês e Grupo (Target vs Control)')
plt.ylabel('Total Amount (R$)')
plt.xlabel('Mês')
plt.legend(title='Grupo')
plt.show()

Observa-se grande numero de outliers, verificar a distribuicao removendo outliers.

In [ ]:
publico_janeiro_dezembro['total_amount_mes'].describe().round(2)


In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
publico_janeiro_dezembro['total_amount_mes'].plot(kind='box', xlim=(0, 500), vert=False, flierprops=red_square, figsize=(16,2))

In [ ]:
distance = 1.5 * (np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 75) - np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 25))
lim_sup=distance + np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 75)

In [ ]:
pb_drop = publico_janeiro_dezembro.drop(
    publico_janeiro_dezembro[
        (publico_janeiro_dezembro['total_amount_mes'] == 0) |
        (publico_janeiro_dezembro['total_amount_mes'] > lim_sup)
    ].index,
    axis=0
)

In [ ]:
pb_drop['total_amount_mes'].describe().round(2)

In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
pb_drop['total_amount_mes'].plot(kind='box', xlim=(0, 400), vert=False, flierprops=red_square, figsize=(16,2))

In [ ]:
df_decil,decil_dict=cria_base_decil_wide(publico_janeiro_dezembro,
    mes_0=12,  
    mes_1=1,  
    col_cliente="customer_id",
    col_mes="order_created_month",
    col_pedidos="num_pedidos_mes",
    col_valor="total_amount_mes",
    col_target="is_target",  
    n_decis=10,
    prefixo_decil="decil",
    prefixo_pedidos="num_pedidos_mes",
    prefixo_valor="total_amount_mes"
)

In [ ]:
df_decil.head()

In [ ]:
save_parquet(df_decil, "../dados/gold", "df_decil.parquet")

In [ ]:
import os
os.makedirs('../dados/gold', exist_ok=True)

decil_dict_json = {k: list(v) for k, v in decil_dict.items()}
with open('../dados/gold/decil_dictionary.json', 'w') as f:
    json.dump(decil_dict_json, f, indent=2)

In [ ]:
decil_summary=gerar_resumo_decis(df_decil)
decil_summary

In [ ]:
os.makedirs('../Resultados', exist_ok=True)

decil_summary.to_csv('../Resultados/decil_summary.csv', index=False)



Numero de pedidos em Janeiro e Dezembro para o quem esteve em ambos meses

In [ ]:

pedidos_hist=resumo_coorte_ativa(publico_janeiro_dezembro,mes_coorte_inicio=12,mes_coorte_fim=1)
pedidos_hist

In [ ]:
pedidos_hist.to_csv('../Resultados/pedidos_hist.csv', index=False)